In [10]:
import bs4 as bs
import urllib.request
import pandas as pd

team_codes = ['arizona-cardinals', 'atlanta-falcons', 'baltimore-ravens', 'buffalo-bills', 'carolina-panthers', 'chicago-bears', 'cincinnati-bengals', 'cleveland-browns', 'dallas-cowboys', 'denver-broncos', 'detroit-lions', 'green-bay-packers', 'houston-texans', 'indianapolis-colts', 'jacksonville-jaguars', 'kansas-city-chiefs', 'los-angeles-chargers', 'los-angeles-rams', 'miami-dolphins', 'minnesota-vikings', 'new-england-patriots', 'new-orleans-saints', 'new-york-giants', 'new-york-jets', 'oakland-raiders', 'philadelphia-eagles', 'pittsburgh-steelers', 'san-francisco-49ers', 'seattle-seahawks', 'tampa-bay-buccaneers', 'tennessee-titans', 'washington-redskins']



In [23]:
teams_dict = {}


def get_player_ids(team):
    # byte_byte = urllib.request.urlopen("https://www.spotrac.com/nfl/cleveland-browns/cap/")
    byte_byte = urllib.request.urlopen("https://www.spotrac.com/nfl/{team}/cap/".format(team=team))
    soups = bs.BeautifulSoup(byte_byte, 'lxml')

    table = soups.find("table", class_="datatable")
    tablesoup = bs.BeautifulSoup(str(table), 'lxml')

    keys = [link.get_text() for link in tablesoup.find_all("a", href=True)]
    vls = [link["href"] for link in tablesoup.find_all("a", href=True)]

    player_keys = []
    link_values = []
    for k in keys:
        if '(' not in k:
            player_keys.append(k)
    # ONE item in the philadelphia eagles page has a value of '-'. So this line of code removes that ONE item.
    if '-' in player_keys:
        player_keys.remove('-')
    # every other value is just "#", so those have to go.
    for v in vls:
        if v != '#':
            link_values.append(v)

    splt_vls = [l.rsplit('/')[-2] for l in link_values]

    # Lots of NFL players have apostrophes in their names, but apostrophes don't work in
    # URLs, so those have to go. This also adds the dash between first/last names.
    players = [v.replace(' ', '-').replace("'", "").lower() for v in player_keys]

    # create a blank dictionary for player urls
    player_ids_dict = {}

    # spotrac_url = "https://www.spotrac.com/nfl/{A}/{B}/cash-earnings/"

    spotrac_player_id = ["{}-{}".format(players[i], splt_vls[i]) for i in range(len(player_keys))]

    for i in range(len(player_keys)):
        player_ids_dict[player_keys[i]] = spotrac_player_id[i]

    if team not in teams_dict:
        teams_dict[team] = player_ids_dict

In [24]:
get_player_ids('new-york-giants')

print(teams_dict)

{'new-york-giants': {'Eli Manning': 'eli-manning-4348', 'Olivier Vernon': 'olivier-vernon-9882', 'Janoris Jenkins': 'janoris-jenkins-9849', 'Nate Solder': 'nate-solder-7732', 'Damon Harrison': 'damon-harrison-10400', 'Saquon Barkley': 'saquon-barkley-25097', 'Odell Beckham, Jr.': 'odell-beckham,-jr.-14421', 'Alec Ogletree': 'alec-ogletree-12310', 'Ereck Flowers': 'ereck-flowers-16733', 'Eli Apple': 'eli-apple-18958', 'Jonathan Stewart': 'jonathan-stewart-4042', 'Rhett Ellison': 'rhett-ellison-9947', 'Kareem Martin': 'kareem-martin-14494', 'Cody Latimer': 'cody-latimer-14466', 'Patrick Omameh': 'patrick-omameh-12961', 'Evan Engram': 'evan-engram-21764', 'Michael Thomas': 'michael-thomas-10443', 'Landon Collins': 'landon-collins-16757', 'Connor Barwin': 'connor-barwin-5680', 'Sterling Shepard': 'sterling-shepard-18989', 'Will Hernandez': 'will-hernandez-25133', 'Mario Edwards': 'mario-edwards-16759', 'Russell Shepard': 'russell-shepard-12926', 'Kerry Wynn': 'kerry-wynn-15027', 'Ray-Ray A

In [20]:
column_headers = ['Year', 'Team', 'Salary', 'Signing_Bonus', 'Roster_Bonus', 'Workout_Bonus', 'Restructure_Bonus', 'Option_Bonus', 'Incentive', 'Total_Cash']

all_players_salaries_dfs_list = []


def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


spotrac_url = "https://www.spotrac.com/nfl/{A}/{B}/cash-earnings/"
# def extract_player_data(player_name, player_earnings_url):
for team_name, player_dict in teams_dict.items():
    for player_name, player_id in player_dict.items():
        player_byte = urllib.request.urlopen(spotrac_url.format(A=team_name, B=player_id))
        soup = bs.BeautifulSoup(player_byte, 'lxml')
        table = soup.find("table", class_="earningstable")
        table_soup = bs.BeautifulSoup(str(table), 'lxml')
        if player_name == "Michael Crabtree":
            print("Fuck Michael Crabtree")
            continue

        for row in table_soup:  # For each row, do the following:
            # 1. Get the text for each table data "td" element (basically, each cell) in the row
            player_list = [td.get_text() for td in row.find_all("td")]

            player_salary_history = []
            # This runs through the
            for i in player_list:
                player_salary_history.append(i)
                if "season" in i:
                    break
            print(player_name)
            print(player_salary_history)
            player_salary_history.pop()

            player_chunks = list(chunks(player_salary_history, 10))

        teams_lst = table_soup.find_all("img", src=True)
        slices = [str(i).rsplit("/", 3)[-2] for i in teams_lst]
        teams = [i.rsplit(".")[-2] for i in slices]

        for i in range(len(player_chunks)):
            player_chunks[i][1] = teams[i]

        player_df = pd.DataFrame(player_chunks, columns=column_headers)

        player_df.insert(0, "Player_Name", player_name)
        player_df_list = player_df.values.tolist()

        all_players_salaries_dfs_list.append(player_df)

salaries_df = pd.concat(all_players_salaries_dfs_list, ignore_index=True)

Joe Flacco
['2008', '', '$295,000', '-', '$1,005,000', '-', '-', '-', '-', '$1,300,000', '2009', '', '$1,535,000', '-', '-', '-', '-', '$4,960,000', '$2,100,000', '$8,595,000', '2010', '', '$2,660,000', '-', '-', '-', '-', '-', '-', '$2,660,000', '2011', '', '$4,485,000', '-', '-', '-', '-', '-', '-', '$4,485,000', '2012', '', '$6,760,000', '-', '-', '-', '-', '-', '-', '$6,760,000', '2013', '', '$1,000,000', '$29,000,000', '-', '-', '-', '-', '-', '$30,000,000', '2014', '', '$6,000,000', '-', '-', '-', '-', '$15,000,000', '-', '$21,000,000', '2015', '', '$4,000,000', '-', '-', '-', '-', '$7,000,000', '-', '$11,000,000', '2016', '', '$4,000,000', '$25,000,000', '-', '-', '-', '-', '-', '$29,000,000', '2017', '', '$6,000,000', '$15,000,000', '-', '-', '-', '-', '-', '$21,000,000', '2018', '', '$12,000,000', '-', '-', '-', '-', '-', '-', '$12,000,000', '11 seasons']
Marshal Yanda
['2007', '', '$285,000', '$502,700', '-', '-', '-', '-', '-', '$787,700', '2008', '', '$370,000', '-', '-', '

Fuck Michael Crabtree
Marlon Humphrey
['2017', '', '$465,000', '$6,756,348', '-', '-', '-', '-', '-', '$7,221,348', '2018', '', '$1,003,522', '-', '-', '-', '-', '-', '-', '$1,003,522', '2 seasons']
James Hurst
['2014', '', '$420,000', '$9,000', '-', '-', '-', '-', '$134,502', '$563,502', '2015', '', '$510,000', '-', '-', '-', '-', '-', '$153,255', '$663,255', '2016', '', '$600,000', '-', '-', '-', '-', '-', '-', '$600,000', '2017', '', '$700,000', '$250,000', '$250,000', '-', '-', '-', '-', '$1,200,000', '2018', '', '$1,000,000', '$5,000,000', '-', '-', '-', '-', '-', '$6,000,000', '5 seasons']
Za'Darius Smith
['2015', '', '$435,000', '$483,152', '-', '-', '-', '-', '$118,224', '$1,036,376', '2016', '', '$525,000', '-', '-', '-', '-', '-', '-', '$525,000', '2017', '', '$615,000', '-', '-', '-', '-', '-', '-', '$615,000', '2018', '', '$1,907,000', '-', '-', '-', '-', '-', '-', '$1,907,000', '4 seasons']
Hayden Hurst
['2018', '', '$480,000', '$6,106,201', '-', '-', '-', '-', '-', '$6,58

Chuck Clark
['2017', '', '$465,000', '$174,807', '-', '-', '-', '-', '-', '$639,807', '2018', '', '$555,000', '-', '-', '-', '-', '-', '-', '$555,000', '2 seasons']
Patrick Ricard
['2017', '', '$465,000', '$6,500', '-', '-', '-', '-', '-', '$471,500', '2018', '', '$555,000', '-', '-', '-', '-', '-', '-', '$555,000', '2 seasons']
Matt Skura
['2016', '', '-', '$7,500', '-', '-', '-', '-', '-', '$7,500', '2016', '', '$117,300', '-', '-', '-', '-', '-', '-', '$117,300', '2017', '', '$410,295', '-', '-', '-', '-', '-', '-', '$410,295', '2017', '', '$14,400', '-', '-', '-', '-', '-', '-', '$14,400', '2018', '', '$555,000', '-', '-', '-', '-', '-', '-', '$555,000', '3 seasons']
Jordan Lasley
['2018', '', '$480,000', '$268,679', '-', '-', '-', '-', '-', '$748,679', '1 season']
Bradley Bozeman
['2018', '', '$480,000', '$118,408', '-', '-', '-', '-', '-', '$598,408', '1 season']
Zach Sieler
['2018', '', '$480,000', '$80,682', '-', '-', '-', '-', '-', '$560,682', '1 season']
Chris Board
['2018', 

In [19]:
column_headers = ['Year', 'Team', 'Salary', 'Signing_Bonus', 'Roster_Bonus', 'Workout_Bonus', 'Restructure_Bonus', 'Option_Bonus', 'Incentive', 'Total_Cash']

all_players_salaries_dfs_list = []


def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


spotrac_url = "https://www.spotrac.com/nfl/baltimore-ravens/michael-crabtree-5964/cash-earnings/"
# def extract_player_data(player_name, player_earnings_url):
player_byte = urllib.request.urlopen(spotrac_url)
soup = bs.BeautifulSoup(player_byte, 'lxml')
table = soup.find("table", class_="earningstable")
table_soup = bs.BeautifulSoup(str(table), 'lxml')

print(soup)

for row in table_soup:  # For each row, do the following:
    # 1. Get the text for each table data "td" element (basically, each cell) in the row
    player_list = [td.get_text() for td in row.find_all("td")]

    player_salary_history = []
    # This runs through the
    for i in player_list:
        player_salary_history.append(i)
        if "season" in i:
            break
    print(player_name)
    print(player_salary_history)

    player_chunks = list(chunks(player_salary_history, 10))

teams_lst = table_soup.find_all("img", src=True)
slices = [str(i).rsplit("/", 3)[-2] for i in teams_lst]
teams = [i.rsplit(".")[-2] for i in slices]

for i in range(len(player_chunks)):
    player_chunks[i][1] = teams[i]

player_df = pd.DataFrame(player_chunks, columns=column_headers)



<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en"> <![endif]--><!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en"> <![endif]--><!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en"> <![endif]--><!--[if IE 9]>    <html class="no-js ie9 oldie" lang="en"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<!-- start:global -->
<meta charset="utf-8"/>
<!-- end:global -->
<!-- start:page title -->
<title>Michael Crabtree Contract Details, Salary Cap Breakdowns, Salaries, Bonuses | Spotrac</title>
<!-- end:page title -->
<!-- start:meta info -->
<meta content="Michael Crabtree, Baltimore Ravens, Baltimore, Ravens, BAL, NFL " name="keywords"/>
<meta content="Michael Crabtree contract and salary cap details, full contract breakdowns, salaries, signing bonus, roster bonus, dead money, and valuations." name="description"/>
<meta content="Spotrac.com" name="author"/>
<meta content="Michael Crabtree" property="og:title"/>


In [ ]:
nyg_dict = {'new-york-giants': {'Eli Manning': 'eli-manning-4348', 'Olivier Vernon': 'olivier-vernon-9882', 'Janoris Jenkins': 'janoris-jenkins-9849', 'Nate Solder': 'nate-solder-7732', 'Damon Harrison': 'damon-harrison-10400', 'Saquon Barkley': 'saquon-barkley-25097', 'Odell Beckham, Jr.': 'odell-beckham,-jr.-14421', 'Alec Ogletree': 'alec-ogletree-12310', 'Ereck Flowers': 'ereck-flowers-16733', 'Eli Apple': 'eli-apple-18958', 'Jonathan Stewart': 'jonathan-stewart-4042', 'Rhett Ellison': 'rhett-ellison-9947', 'Kareem Martin': 'kareem-martin-14494', 'Cody Latimer': 'cody-latimer-14466', 'Patrick Omameh': 'patrick-omameh-12961', 'Evan Engram': 'evan-engram-21764', 'Michael Thomas': 'michael-thomas-10443', 'Landon Collins': 'landon-collins-16757', 'Connor Barwin': 'connor-barwin-5680', 'Sterling Shepard': 'sterling-shepard-18989', 'Will Hernandez': 'will-hernandez-25133', 'Mario Edwards': 'mario-edwards-16759', 'Russell Shepard': 'russell-shepard-12926', 'Kerry Wynn': 'kerry-wynn-15027', 'Ray-Ray Armstrong': 'ray-ray-armstrong-13147', 'Dalvin Tomlinson': 'dalvin-tomlinson-21796', 'Zak DeOssie': 'zak-deossie-4314', 'John Jenkins': 'john-jenkins-12363', 'Nathan Stupar': 'nathan-stupar-10051', 'B.J. Goodson': 'b.j.-goodson-19063', 'Lorenzo Carter': 'lorenzo-carter-25165', 'B.J. Hill': 'b.j.-hill-25168', 'John Greco': 'john-greco-3144', 'Scott Simonson': 'scott-simonson-15044', 'Wayne Gallman': 'wayne-gallman-21882', 'Kyle Lauletta': 'kyle-lauletta-25209', 'B.W. Webb': 'b.w.-webb-12395', 'Kaelin Clay': 'kaelin-clay-16910', 'Riley Dixon': 'riley-dixon-19182', 'Antonio Hamilton': 'antonio-hamilton-19547', 'Mike Jordan': 'mike-jordan-19281', 'Spencer Pulley': 'spencer-pulley-19563', 'Curtis Riley': 'curtis-riley-17354', 'Chad Wheeler': 'chad-wheeler-22311', 'Donte Deayon': 'donte-deayon-19225', 'Jon Halapio': 'jon-halapio-14590', 'Aldrick Rosas': 'aldrick-rosas-19304', 'Shane Smith': 'shane-smith-22316', 'Alex Tanney': 'alex-tanney-10360', 'Sean Chandler': 'sean-chandler-25571', 'Evan Brown': 'evan-brown-25809', 'Tae Davis': 'tae-davis-25813', 'Kamrin Moore': 'kamrin-moore-25290'}}

column_headers = ['Year', 'Team', 'Salary', 'Signing_Bonus', 'Roster_Bonus', 'Workout_Bonus', 'Restructure_Bonus', 'Option_Bonus', 'Incentive', 'Total_Cash']


def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


all_players_salaries_dfs_list = []
player_errors = []
    
spotrac_url = "https://www.spotrac.com/nfl/{A}/{B}/cash-earnings/"

# def extract_player_data(player_name, player_earnings_url):
for team_name, player_dict in nyg_dict.items():
    for player_name, player_id in player_dict.items():
        try:
            player_byte = urllib.request.urlopen(spotrac_url.format(A=team_name, B=player_id))
            soup = bs.BeautifulSoup(player_byte, 'lxml')
            table = soup.find("table", class_="earningstable")
            table_soup = bs.BeautifulSoup(str(table), 'lxml')
            if player_name == "Michael Crabtree":
                print("Fuck Michael Crabtree")
                continue

            for row in table_soup:  # For each row, do the following:
                # 1. Get the text for each table data "td" element (basically, each cell) in the row
                player_list = [td.get_text() for td in row.find_all("td")]

                player_salary_history = []
                # This runs through the
                for i in player_list:
                    player_salary_history.append(i)
                    if "season" in i:
                        break
                player_salary_history.pop()

                player_chunks = list(chunks(player_salary_history, 10))

            teams_lst = table_soup.find_all("img", src=True)
            slices = [str(i).rsplit("/", 3)[-2] for i in teams_lst]
            teams = [i.rsplit(".")[-2] for i in slices]

            for i in range(len(player_chunks)):
                player_chunks[i][1] = teams[i]

            player_df = pd.DataFrame(player_chunks, columns=column_headers)

            player_df.insert(0, "Player_Name", player_name)
            player_df.insert(1, "Spotrac_ID", player_id)
            player_df_list = player_df.values.tolist()
            print(player_df_list[0])
            all_players_salaries_dfs_list.append(player_df)
        except Exception as e:
            # see reference_scraper.py line 148-152
            player_errors = [player_name, e]
            errors_list.append(player_errors)

print(player_errors)

In [15]:
from salary_scraper2001plus import current_player_salaries

In [17]:
current_player_salaries.head()

,Player_Name,Spotrac_ID,Year,Team,Salary,Signing_Bonus,Roster_Bonus,Workout_Bonus,Restructure_Bonus,Option_Bonus,Incentive,Total_Cash
0,Larry Fitzgerald,larry-fitzgerald-966,2004,arizona2,"$1,050,000","$7,500,000",-,-,-,-,-,"$8,550,000"
1,Larry Fitzgerald,larry-fitzgerald-966,2005,arizona2,"$305,000",-,-,-,-,"$5,030,000",-,"$5,335,000"
2,Larry Fitzgerald,larry-fitzgerald-966,2006,arizona2,"$3,317,500",-,-,-,-,-,-,"$3,317,500"
3,Larry Fitzgerald,larry-fitzgerald-966,2007,arizona2,"$3,580,000",-,-,-,-,-,-,"$3,580,000"
4,Larry Fitzgerald,larry-fitzgerald-966,2008,arizona2,"$2,100,000","$15,000,000",-,-,-,-,-,"$17,100,000"


In [28]:
current_player_salaries.Total_Cash = current_player_salaries.Total_Cash.str.replace('$', '')
current_player_salaries.Total_Cash = current_player_salaries.Total_Cash.str.replace(',', '')

In [34]:
current_player_salaries.Total_Cash = current_player_salaries.Total_Cash.str.replace('$', '')
current_player_salaries.Total_Cash = current_player_salaries.Total_Cash.str.replace(',', '')

current_player_salaries.Salary = current_player_salaries.Salary.str.replace('$', '')
current_player_salaries.Salary = current_player_salaries.Salary.str.replace(',', '')
current_player_salaries.Salary = current_player_salaries.Salary.str.replace('-', '')

current_player_salaries.Signing_Bonus = current_player_salaries.Signing_Bonus.str.replace('$', '')
current_player_salaries.Signing_Bonus = current_player_salaries.Signing_Bonus.str.replace(',', '')
current_player_salaries.Signing_Bonus = current_player_salaries.Signing_Bonus.str.replace('-', '')

current_player_salaries.Roster_Bonus = current_player_salaries.Roster_Bonus.str.replace('$', '')
current_player_salaries.Roster_Bonus = current_player_salaries.Roster_Bonus.str.replace(',', '')
current_player_salaries.Roster_Bonus = current_player_salaries.Roster_Bonus.str.replace('-', '')

current_player_salaries.Workout_Bonus = current_player_salaries.Workout_Bonus.str.replace('$', '')
current_player_salaries.Workout_Bonus = current_player_salaries.Workout_Bonus.str.replace(',', '')
current_player_salaries.Workout_Bonus = current_player_salaries.Workout_Bonus.str.replace('-', '')

current_player_salaries.Restructure_Bonus = current_player_salaries.Restructure_Bonus.str.replace('$', '')
current_player_salaries.Restructure_Bonus = current_player_salaries.Restructure_Bonus.str.replace(',', '')
current_player_salaries.Restructure_Bonus = current_player_salaries.Restructure_Bonus.str.replace('-', '')

current_player_salaries.Option_Bonus = current_player_salaries.Option_Bonus.str.replace('$', '')
current_player_salaries.Option_Bonus = current_player_salaries.Option_Bonus.str.replace(',', '')
current_player_salaries.Option_Bonus = current_player_salaries.Option_Bonus.str.replace('-', '')

current_player_salaries.Incentive = current_player_salaries.Incentive.str.replace('$', '')
current_player_salaries.Incentive = current_player_salaries.Incentive.str.replace(',', '')
current_player_salaries.Incentive = current_player_salaries.Incentive.str.replace('-', '')

In [35]:
current_player_salaries

,Player_Name,Spotrac_ID,Year,Team,Salary,Signing_Bonus,Roster_Bonus,Workout_Bonus,Restructure_Bonus,Option_Bonus,Incentive,Total_Cash
0,Larry Fitzgerald,larry-fitzgerald-966,2004,arizona2,1050000,7500000,,,,,,8550000
1,Larry Fitzgerald,larry-fitzgerald-966,2005,arizona2,305000,,,,,5030000,,5335000
2,Larry Fitzgerald,larry-fitzgerald-966,2006,arizona2,3317500,,,,,,,3317500
3,Larry Fitzgerald,larry-fitzgerald-966,2007,arizona2,3580000,,,,,,,3580000
4,Larry Fitzgerald,larry-fitzgerald-966,2008,arizona2,2100000,15000000,,,,,,17100000
5,Larry Fitzgerald,larry-fitzgerald-966,2009,arizona2,1600000,,3150000,,,5000000,,9750000
6,Larry Fitzgerald,larry-fitzgerald-966,2010,arizona2,6150000,,,,,,,6150000
7,Larry Fitzgerald,larry-fitzgerald-966,2011,arizona2,2000000,10000000,8000000,,,,,20000000
8,Larry Fitzgerald,larry-fitzgerald-966,2012,arizona2,5000000,,6000000,250000,,15000000,,26250000
9,Larry Fitzgerald,larry-fitzgerald-966,2013,arizona2,5000000,,,250000,,,,5250000
